In [1]:
import numpy as np

In [2]:
class Node():
    def __init__(self, pos, dist):
        self.pos = pos
        self.dist = dist
        self.neighbours = {}

    def __repr__(self):
        return f'Node at pos {self.pos}'

    def add_neighbour(self, n, dist):
        edge = Edge(dist, self, n)
        self.neighbours[n] = edge
        n.neighbours[self] = edge

class Edge():
    def __init__(self, length, n1, n2):
        self.n1 = n1
        self.n2 = n2
        self.length = length

    def __repr__(self):
        return f'{self.n1} -> {self.n2}: {self.length}'
    
dir_map = {
    'N': (0, -1),
    'S': (0, 1),
    'W': (-1, 0),
    'E': (1, 0)
}

In [3]:
# with open("test_input.txt") as fh:
with open("input.txt") as fh:
# with open("test2.txt") as fh:
    bits = [[int(y) for y in x.strip().split(',')] for x in fh.readlines()]
    if len(bits) == 25:
        grid = 6
    else:
        grid = 70
    prearr = np.full((grid + 1, grid + 1), '.', dtype=str)

In [4]:
def make_map(arr, bits, time):
    bits = bits[:time]
    for bit in bits:
        x, y = bit
        arr[y][x] = '#'
    return arr


In [5]:
from collections import deque

In [6]:
def build_nodes(arr):
    all_node_pos = {}
    for y in range(arr.shape[0]):
        for x in range(arr.shape[1]):
            if arr[y][x] == '#':
                continue
            all_node_pos[(x, y)] = Node((x, y), np.inf)

    for node_pos, node in all_node_pos.items():
        for dir in ['N', 'S', 'W', 'E']:
            tile = '.'
            dist = 0
            nx, ny = node_pos[0] + dir_map[dir][0], node_pos[1] + dir_map[dir][1]
            if 0 > nx  or nx >= arr.shape[0] or 0 > ny or ny >= arr.shape[1]:
                continue
            tile = arr[ny][nx]
            if tile == '#':
                continue
            else:
                node.add_neighbour(all_node_pos[(nx, ny)], 1)
    return all_node_pos

In [7]:
def dijkstra(all_node_pos, start):
    for node in all_node_pos.values():
        node.dist = np.inf
        node.last_dir = ' '
        node.prev = set()

    start = (0, 0)

    node = all_node_pos[start]
    node.dist = 0
    queue = deque(all_node_pos.values())

    while len(queue) > 0:
        node = sorted(queue, key=lambda x: x.dist)[0]
        queue.remove(node)
        dist = node.dist
        for neighbour, edge in node.neighbours.items():
            score = dist + edge.length 
            if score < neighbour.dist:
                neighbour.prev = set()
                neighbour.prev.add(node)
                neighbour.dist = score
                node.prev.add(neighbour)
            elif score == neighbour.dist:
                neighbour.prev.add(node)
    return all_node_pos
    

In [8]:
def check_pos(arr, i):
    arr = make_map(prearr.copy(), bits, i)
    all_node_pos = build_nodes(arr)
    all_node_pos = dijkstra(all_node_pos, (0, 0))
    return all_node_pos

In [9]:
all_node_pos = check_pos(prearr.copy(), 1024)
all_node_pos[(grid, grid)].dist

408

In [ ]:
L, R = 0, len(bits)
res = {}
mid = (L + R) // 2
prev_d = None
while True:
    i = mid
    all_node_pos = check_pos(prearr.copy(), i)
    if all_node_pos[(grid, grid)].dist == np.inf:
        if i - 1 in res and res[i - 1][(grid, grid)].dist != np.inf:
            print(f"Found Cut Point: {i}")
            res[i] = all_node_pos
            break
        R = mid
        mid = (L + R) // 2
    else:
        if i + 1 in res and res[i + 1][(grid, grid)].dist == np.inf:
            print(f"Found Cut Point: {i + 1}")
            res[i] = all_node_pos
            break
        L = mid
        mid = (L + R) // 2
    res[i] = all_node_pos

Found Cut Point: 2893


In [11]:
bits[2892]

[45, 16]